In [2]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('./cvss_2022_2024_X_train-attack-vector.csv')
y_train = pd.read_csv('./cvss_2022_2024_y_train-attack-vector.csv')

X_test = pd.read_csv('./cvss_2022_2024_X_test-attack-vector.csv')
y_test = pd.read_csv('./cvss_2022_2024_y_test-attack-vector.csv')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train['cvssv3_attack_vector'].value_counts(dropna=False) / y_train.shape[0])
print(y_test['cvssv3_attack_vector'].value_counts(dropna=False) / y_test.shape[0])
print(X_train.info())
print(y_train.info())

(142192, 1)
(35548, 1)
(142192, 1)
(35548, 1)
cvssv3_attack_vector
ADJACENT_NETWORK    0.250225
NETWORK             0.250077
PHYSICAL            0.249887
LOCAL               0.249810
Name: count, dtype: float64
cvssv3_attack_vector
LOCAL               0.250760
PHYSICAL            0.250450
NETWORK             0.249691
ADJACENT_NETWORK    0.249100
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142192 entries, 0 to 142191
Data columns (total 1 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   english_description  142192 non-null  object
dtypes: object(1)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142192 entries, 0 to 142191
Data columns (total 1 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   cvssv3_attack_vector  142192 non-null  object
dtypes: object(1)
memory usage: 1.1+ MB
None


In [3]:
label_column_name = "cvssv3_attack_vector"
train_labels = y_train.loc[:, label_column_name]
test_labels = y_test.loc[:, label_column_name]


from sklearn.preprocessing import LabelEncoder
import pickle

le = LabelEncoder()
le.fit(train_labels)
print(le.classes_)

with open("../labels/attack_vector_label.txt", "wb") as f:
    pickle.dump(le.classes_, f)

NUM_CLASSES = len(le.classes_)
print(NUM_CLASSES)

encoded_train_labels = le.transform(train_labels)
encoded_test_labels = le.transform(test_labels)

print(train_labels[:10], encoded_train_labels[:10])
print(len(X_train), len(train_labels), len(X_test), len(test_labels))

['ADJACENT_NETWORK' 'LOCAL' 'NETWORK' 'PHYSICAL']
4
0               LOCAL
1    ADJACENT_NETWORK
2               LOCAL
3            PHYSICAL
4            PHYSICAL
5               LOCAL
6             NETWORK
7    ADJACENT_NETWORK
8            PHYSICAL
9               LOCAL
Name: cvssv3_attack_vector, dtype: object [1 0 1 3 3 1 2 0 3 1]
142192 142192 35548 35548


In [4]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-small')

d:\my-proj\ML - Proje\CVSS-Prediction-BERT_small\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_encodings = tokenizer(X_train.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)

In [6]:
import torch

class CVEDataset(torch.utils.data.Dataset):
    def __init__(self, X, encodings, labels, encoded_labels):
        self.texts = X.loc[:,"english_description"].tolist()
        self.encodings = encodings
        self.labels = labels.tolist()
        self.encoded_labels = encoded_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['text_labels'] = self.labels[idx]
        item['encoded_labels'] = torch.tensor(self.encoded_labels[idx])
        item['vulnerability_description'] = self.texts[idx]
        
        return item

    def __len__(self):
        return len(self.labels)


In [7]:
train_dataset = CVEDataset(X_train, train_encodings, train_labels, encoded_train_labels)
test_dataset = CVEDataset(X_test, test_encodings, test_labels, encoded_test_labels)

In [8]:
import torch
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('../models/bert-small-vulnerability_attack_vector-classification')
model.to(device) 
model.eval()

test_loader = DataLoader(test_dataset, batch_size=16)


num_correct = 0 
num_examples = 0
test_loss = 0
predicted_labels_list = []
predicted_labels_score_list = []
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['encoded_labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    test_loss += loss.data.item() * input_ids.size(0)
    predicted_labels = torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1]
    predicted_labels_list.extend(predicted_labels.tolist())
    predicted_labels_score = torch.max(F.softmax(outputs.logits, dim=1), dim=1)[0]
    predicted_labels_score_list.extend(predicted_labels_score.tolist())
    correct = torch.eq(predicted_labels, labels)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
test_loss /= len(test_loader.dataset)

        
print('Test Loss: {}, Test Accuracy = {}'.format(test_loss, num_correct / num_examples))
print('predicted labels:', predicted_labels_list, 'scores:', predicted_labels_score_list)

Test Loss: 0.0699526821438275, Test Accuracy = 0.9790986834702374
predicted labels: [2, 2, 1, 3, 3, 1, 0, 1, 3, 2, 2, 0, 3, 3, 1, 1, 0, 2, 2, 0, 3, 3, 2, 1, 0, 1, 3, 1, 3, 2, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 3, 2, 0, 1, 3, 0, 1, 2, 1, 0, 1, 1, 0, 2, 3, 2, 1, 3, 1, 3, 3, 3, 0, 3, 0, 1, 0, 3, 1, 0, 3, 2, 0, 0, 0, 3, 0, 1, 2, 1, 2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 3, 3, 0, 3, 2, 0, 2, 3, 1, 2, 0, 2, 1, 2, 3, 0, 1, 1, 0, 3, 0, 1, 1, 2, 1, 1, 0, 2, 1, 1, 3, 3, 0, 2, 2, 1, 3, 2, 0, 0, 1, 3, 0, 0, 0, 2, 3, 1, 0, 1, 0, 0, 1, 2, 1, 1, 3, 0, 3, 2, 3, 1, 0, 1, 1, 3, 1, 2, 0, 0, 3, 2, 0, 3, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 3, 0, 0, 3, 3, 3, 3, 0, 3, 3, 2, 1, 0, 3, 0, 3, 3, 1, 1, 0, 2, 0, 3, 0, 0, 2, 2, 2, 3, 2, 1, 1, 1, 2, 3, 0, 1, 2, 2, 2, 0, 1, 3, 1, 0, 3, 1, 0, 1, 0, 0, 1, 2, 1, 3, 2, 2, 0, 3, 0, 0, 3, 2, 0, 3, 1, 3, 1, 1, 0, 2, 0, 0, 2, 1, 1, 2, 1, 0, 0, 2, 2, 2, 3, 3, 1, 1, 1, 0, 3, 0, 2, 2, 1, 2, 2, 0, 2, 2, 1, 2, 0, 3, 1, 1, 3, 1, 1, 0, 1, 2, 2, 2, 0, 2, 0, 1, 0, 1, 2, 2, 2, 1, 0, 2, 3, 2, 1, 0, 2, 

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score

# Assuming test_dataset and predicted_labels_list are defined
y_true = test_dataset[:]['encoded_labels'].numpy()
y_pred = np.array(predicted_labels_list)

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
balanced_accuracy = balanced_accuracy_score(y_true, y_pred)


# Print results
print("Accuracy: ", accuracy)
print("Balanced Accuracy: ", balanced_accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Accuracy:  0.9790986834702374
Balanced Accuracy:  0.979100934016057
Precision:  0.9791861958168625
Recall:  0.9790986834702374
F1 Score:  0.9790511002300846

Confusion Matrix:
[[8828   13    3   11]
 [  33 8683  174   24]
 [  55  391 8417   13]
 [  26    0    0 8877]]
